### Scrapear Scrapear Amazon

Vamos a sacar todos los datos de cada bloque de producto en nuestra busqueda de amazon. En este caso de productos Lego y vamos a sacar el nombre del producto, el precio, el numero de estrellas y el link, sin mostrar los patrocinados.

Primero importamos lo que necesitamos y configuramos nuestro driver de Selenium

In [42]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
import time
import random
import selenium.webdriver.common.keys as Keys # para simular teclas
import pandas as pd
from bs4 import BeautifulSoup as bs

#Opciones de chrome
opciones = webdriver.ChromeOptions()
#opciones.add_argument('--start-maximized')
opciones.add_argument('window-size=1850,1000')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# guardar las cookies
# opciones.add_argument('user-data-dir=cookies')    # mantiene las cookies
opciones.add_argument('--incognito') # modo incognito

url = 'https://www.amazon.es/'

In [44]:
# Abrimos nuestro driver cargandole nuestras opciones y cerrando las cookies
# despues de un tiempo prudencial para un humano
driver = webdriver.Chrome(options=opciones)

driver.get(url)
 
time.sleep(1)
driver.find_element(By.ID, 'sp-cc-rejectall-link').click()

In [45]:
# Hacemos lo del ninja que le gusta a Victor

# Seleccionamos el elemento de input de texto para busquedas de Amazon con su XPATH
cuadro_busqueda = driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]')

#MODO NINJA ACTIVADO
#Queremos escribir juguetes lego, pero vamos a hacerlo letra a letra como lo haría un humano
time.sleep(1)

#Borramos la busqueda anterior:
cuadro_busqueda.clear()
time.sleep(2)
#Y ahora vamos a escribir letra a letra:
termino_busqueda = 'juguetes lego'
for letra in termino_busqueda:
    cuadro_busqueda.send_keys(letra)
    time.sleep(random.uniform(0.1,0.25))
cuadro_busqueda.submit()

In [46]:
#Sacamos listado de los productos de la página:
productos = driver.find_elements(By.CLASS_NAME, 'a-section.a-spacing-base')


In [47]:
#Le pasamos el listado de productos a Bs4
soup = bs(driver.page_source, 'html.parser')
# Creamos la lista de divs con las clases que queremos, pero que contienen todos, incluyendo los promocionales
productos_raw = soup.find_all('div', class_='a-section a-spacing-base')

productos = []
# Mostramos la lista de productos y la guardamos en nuestra lista filtrada sin promocionados
for producto in productos_raw:
    #Filtramos los productos por los que no son promocionados
    if "Patrocinado" not in producto.text:
        print(producto.text)
        print(producto.find('h2').text)
        print(producto.find('i').text)
        print(len(producto.find_all('span', class_='a-price')))
        print(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.')+"€")
        # Los añadimos a la verdadera lista de productos
        productos.append(producto)

Opción Amazonpara "juguetes lego"  LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  4,8 de 5 estrellas 30.640  1 mil+ comprados el mes pasado24,01 €24,01€ PVPR: 29,99 €29,99€Entrega GRATIS el lun, 30 de sept en tu primer pedidoEntrega más rápida el sáb, 28 de sept Edades: 4 años y más
LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  
4,8 de 5 estrellas
2
24.01€
Más vendidoen Figuras de dinosaurios y criaturas prehistóricas de juguete para niños  LEGO 31058 Creator Grandes Dinosaurios, Regalo de Reyes para Niños y Niñas de 7 Años o Más, Maqueta 3en1 de Pterodáctilo, Triceratops y T-Rex, Figura de Animales de Juguete de Parque Jurásico  4,7 de 5 estrellas 3.144  600+ comprados el mes pasado1

In [49]:
# Al final ha sido mas trabajo de BS que de Selenium...pero bueno, creo que esta
# bastante generalizado.

info_productos = []
for i, producto in enumerate(productos):
    dicc_prod = {}
    
    #Link del producto:
    prefijo_url = 'https://www.amazon.es'
    link_parcial = producto.find('a').attrs['href']
    link_producto = prefijo_url+link_parcial
    dicc_prod['link_producto'] = link_producto
    print(f"El producto {i} tiene el siguiente link:\n{link_producto}")
    
   
    #Nombre producto
    nombre = producto.find('h2').text
    dicc_prod['nombre'] = nombre
    print(f"El siguiente nombre:\n{nombre}")
    
    
    #Estrellas
    valoracion = float(producto.find('i').text.split(' ')[0].replace(',','.'))
    dicc_prod['valoracion'] = valoracion
    print(f"Las siguientes estrellas: {valoracion}")
    

    #Precio
    precio = float(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.'))
    dicc_prod['precio'] = precio
    print(f"El siguiente precio: {precio}€\n")
            
    info_productos.append(dicc_prod)
    

El producto 0 tiene el siguiente link:
https://www.amazon.es/LEGO-Classic-10696-ladrillos-creativos/dp/B00NVDP3ZU/ref=ice_ac_b_dpb?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1K2VCDZV8TMS3&dib=eyJ2IjoiMSJ9.JeseBEYIk1Lq23zhgXFBeLvR6IAQsidBwkDGe4FZrAkwxvV0nuk8HBhBHV9v0ErdfgH8hu9zwbmW1hAbmX21SalCQ89UhT9dhTVetx9a5zBdU_g3ukVERk-V4sMAFa3_bfE3RhQWV_FjiZXKPuryM6Efn8lQqDUDcyLUgSxz7uow5sw_LRepOvO_EwRp4QO0fui9Dafqi4wd4TKNpNaskf4WPD2eH_Y_q9DPYGcXX7EBN0UjCHGIVoxzA0AolW-Zz03Ms1oFgxf4daZrhLaEo1gef6wCanyW2jaVvtBiaiw.Vkir4lx6IVN943wMUr3mh7pPLbUXwLQLZX5Wr2Rl8dg&dib_tag=se&keywords=juguetes+lego&qid=1727376837&sprefix=juguetes+lego%2Caps%2C103&sr=8-2
El siguiente nombre:
LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  
Las siguientes estrellas: 4.8
El siguiente precio: 24.01€

El producto 1 tiene el siguiente link:
https://www.amazon.es/gp/bestsellers

In [50]:
# Creamos nuestro dataframe con los legos scrapeados
df_amazon = pd.DataFrame(info_productos)
df_amazon

,link_producto,nombre,valoracion,precio
0,https://www.amazon.es/LEGO-Classic-10696-ladri...,LEGO Classic Caja de Ladrillos Creativos Media...,4.8,24.01
1,https://www.amazon.es/gp/bestsellers/toys/1452...,"LEGO 31058 Creator Grandes Dinosaurios, Regalo...",4.7,13.64
2,https://www.amazon.es/LEGO-Marvel-Persecuci%C3...,LEGO Marvel Persecución en Moto: Spider-Man vs...,4.6,9.99
3,https://www.amazon.es/LEGO-76990-Sonic-Hedgeho...,LEGO Sonic The Hedgehog Sonic: Desafío de la E...,4.8,29.99
4,https://www.amazon.es/LEGO-60373-Bomberos-Flot...,LEGO City Lancha de Rescate de Bomberos y Zodi...,4.8,19.99
5,https://www.amazon.es/LEGO-21247-Casa-Ajolote-...,"LEGO Minecraft Set La Casa-Ajolote, Base Subma...",4.8,26.99
6,https://www.amazon.es/LEGO-10969-Construcci%C3...,"LEGO Duplo Camión de Bomberos de Juguete, Jugu...",4.8,29.99
7,https://www.amazon.es/LEGO-76226-Juguete-de-co...,LEGO 76226 Marvel Figura de Spider-Man Articul...,4.8,29.99
8,https://www.amazon.es/LEGO-Calientes-Accesorio...,LEGO Friends Camión de Perritos Calientes Food...,4.8,19.99
9,https://www.amazon.es/LEGO-60304-Carretera-Con...,"LEGO City Bases de Carretera, Juguetes para Ni...",4.7,19.99


In [51]:
# Celda para cerrarlo, por si tenemos que reiniciar cositas
driver.quit()

### Screapear Amazon Version 2.0
Codigo que realiza cualquier busqueda en amazon, y devuelve un JSON con el nombre de la busqueda mas "_Amazon" de todos los elementos encontrados al realizar esa busqueda. Esta podra ademas añadir mas de una pagina y la opcion para incluir ofertas patrocinadas, si o no.



Una sola celda, un solo click(O Ctrl + Enter)

In [40]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
import time
import random
import selenium.webdriver.common.keys as Keys # para simular teclas
import pandas as pd
from bs4 import BeautifulSoup as bs

# Funciones
def guardar_json(df, ruta_json):
  '''
  Esta funcion recibe como parametros un dataframe y una ruta
  y crea un fichero json con los datos del dataframe en esa ruta
  '''
  # Creo que no necesita return...pero ya me regañaran si eso
  df.to_json(ruta_json, orient='records', lines=True)


#Opciones de chrome
opciones = webdriver.ChromeOptions()
#opciones.add_argument('--start-maximized')
opciones.add_argument('window-size=1850,1000')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# guardar las cookies
# opciones.add_argument('user-data-dir=cookies')    # mantiene las cookies
opciones.add_argument('--incognito') # modo incognito

url = 'https://www.amazon.es/'

# Abrimos nuestro driver cargandole nuestras opciones y cerrando las cookies
# despues de un tiempo prudencial para un humano
driver = webdriver.Chrome(options=opciones)

driver.get(url)
 
time.sleep(1)
driver.find_element(By.ID, 'sp-cc-rejectall-link').click()

termino_busqueda = input("¿Que buscamos en Amazon? ")

# Seleccionamos el elemento de input de texto para busquedas de Amazon con su XPATH
cuadro_busqueda = driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]')
# Empezamos la parafernalia de escribir nuestra busqueda
time.sleep(1)
#Borramos la busqueda anterior por si hay
cuadro_busqueda.clear()
time.sleep(2)
#Y ahora vamos a escribir letra a letra:
for letra in termino_busqueda:
    cuadro_busqueda.send_keys(letra)
    time.sleep(random.uniform(0.1,0.25))
cuadro_busqueda.submit()

soup = bs(driver.page_source, 'html.parser')

# Vale asi no funciona...tengo que hacer algo que sea valido para todos
# productos_raw = soup.find_all('div', class_='a-section a-spacing-base')

# Vamos a extraer las clases del div como lista y verificar si empiezan 
# con las clases que queremos a-section a-spacing-base que si contienen todas
# los bloques de elementos en todas las paginas
productos_raw = []

for div in soup.find_all("div"):
    class_list = div.get("class", [])
    # Convertimos la lista de clases a un string para poder hacer la comparación
    # Quitamos las certificaciones
    if "s-pc-certification" not in " ".join(class_list):
        if " ".join(class_list[:2]) == "a-section a-spacing-base":
            productos_raw.append(div)

# A ver como me queda esto
# for producto in productos_raw:
#    print(producto)
# Genial lo esta haciendo de puta madre, continuamos para bingo

productos = []
# Mostramos la lista de productos y la guardamos en nuestra lista filtrada sin promocionados
for producto in productos_raw:
    if "Patrocinado" not in producto.text and "certificado" not in producto.text:
        # print(producto.text)
        # print(producto.find('h2').text)
        # print(producto.find('i').text)
        # print(len(producto.find_all('span', class_='a-price')))
        # print(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.')+"€")
        productos.append(producto)
        
# Y ahora creamos nuestra lista de diccionarios, donde vamos a almacenar nuestros datos.
info_productos = []
for i, producto in enumerate(productos):
    dicc_prod = {}
    
    #Link del producto:
    prefijo_url = 'https://www.amazon.es'
    link_parcial = producto.find('a').attrs['href']
    link_producto = prefijo_url+link_parcial
    dicc_prod['link_producto'] = link_producto
    print(f"El producto {i} tiene el siguiente link:\n{link_producto}")
    
   
    #Nombre producto
    nombre = producto.find('h2').text
    dicc_prod['nombre'] = nombre
    print(f"El siguiente nombre:\n{nombre}")
    
    #Estrellas
    valoracion = float(producto.find('i').text.split(' ')[0].replace(',','.'))
    dicc_prod['valoracion'] = valoracion
    print(f"Las siguientes estrellas: {valoracion}")
    

    #Precio
    precio = float(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.'))
    dicc_prod['precio'] = precio
    print(f"El siguiente precio: {precio}€\n")
            
    info_productos.append(dicc_prod)

df_amazon = pd.DataFrame(info_productos)

archivo_json = input("Nombre del Json donde vamos a guardar el Resultado: ")
ruta_json = 'archivos' + archivo_json + '.json'
guardar_json(df_amazon, ruta_json)

El producto 0 tiene el siguiente link:
https://www.amazon.es/Reebok-Trainer-Zapatillas-Deporte-Hombre/dp/B0CB7SPSMK/ref=sr_1_5?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2MB8WREQDRACF&dib=eyJ2IjoiMSJ9.co_eeB9QNhZE3rR0loRPVW4CZ9Adfq3XA62lWWYQ8DHk7A-u8GFk9yuhO0lzgz-mfKujdnfqHYdtQJ_7QmuxTfPN3zRUwQWL-mgbOM_6UA1sl_NMI_VvW51eJx_NTN-AEFN4_21Zj3utSD3HrXavfNcT6SwHsHGxWA1GhGXS_RZRGOORaiVJPDiy_FmxA1kwmOnxsPMByWWmcVvxI1qVKyUcrjjwjwIek5HUlaifBpW7O4fQfg4eyWLC08N2neANOZ8pKqID6GnSiPRptsRiQkftRhOAJprPXga3AkiTUH8._86HSIadX1zTKx053tn4hfSiuVprowSLiWLVPJS0x8k&dib_tag=se&keywords=zapatillas&qid=1727376644&sprefix=zapatillas%2Caps%2C120&sr=8-5
El siguiente nombre:
Reebok
Las siguientes estrellas: 4.3
El siguiente precio: 33.0€

El producto 1 tiene el siguiente link:
https://www.amazon.es/RockDove-Invierno-Hombres-Gris-Oscuro/dp/B01MR32LQT/ref=ice_ac_b_dpb?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2MB8WREQDRACF&dib=eyJ2IjoiMSJ9.co_eeB9QNhZE3rR0loRPVW4CZ9Adfq3XA62lWWYQ8DHk7A-u8GFk9yuhO0lzgz-mfKujdnf

AttributeError: 'NoneType' object has no attribute 'text'

In [41]:
print(producto.prettify())

<div class="a-section a-spacing-base a-text-center">
 <div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-v2fl5pkubaqu126k6zseo6li6q" data-cy="image-container" style="padding-top: 30px !important;">
  <div class="s-image-padding">
   <span class="rush-component" data-component-type="s-product-image" data-render-id="r39jmgfgsut4vt22v2ueluvtots" data-version-id="v2fl5pkubaqu126k6zseo6li6q">
    <a class="a-link-normal s-no-outline" href="/Geox-%C3%81lbumes-Zapatillas-Beige-Amarillo/dp/B0C66J3RP1/ref=sr_1_39?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&amp;crid=2MB8WREQDRACF&amp;dib=eyJ2IjoiMSJ9.co_eeB9QNhZE3rR0loRPVW4CZ9Adfq3XA62lWWYQ8DHk7A-u8GFk9yuhO0lzgz-mfKujdnfqHYdtQJ_7QmuxTfPN3zRUwQWL-mgbOM_6UA1sl_NMI_VvW51eJx_NTN-AEFN4_21Zj3utSD3HrXavfNcT6SwHsHGxWA1GhGXS_RZRGOORaiVJPDiy_FmxA1kwmOnxsPMByWWmcVvxI1qVKyUcrjjwjwIek5HUlaifBpW7O4fQfg4eyWLC08N2n

In [31]:
# Celda para cerrarlo, por si tenemos que reiniciar cositas
driver.quit()

### Pruebas y Borradores

In [6]:
#Sacamos listado de los productos de la página:
productos = driver.find_elements(By.CLASS_NAME, 'a-section.a-spacing-base')
productos.__len__()

69

In [ ]:
blorgh = driver.find_element(By.CSS_SELECTOR, '[placeholder = "Your comment"]')

In [39]:
#Le pasamos el listado de productos a Bs4
from bs4 import BeautifulSoup as bs
soup = bs(driver.page_source, 'html.parser')
productos_raw = soup.find_all('div', class_='a-section a-spacing-base')

productos = []
# Mostramos la lista de productos y la guardamos en nuestra lista filtrada sin promocionados
for producto in productos_raw:
    if "Patrocinado" not in producto.text:
        print(producto.text)
        print(producto.find('h2').text)
        print(producto.find('i').text)
        print(len(producto.find_all('span', class_='a-price')))
        print(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.')+"€")
        productos.append(producto)
   # else:
      #  productos.remove(producto)

    

Opción Amazonpara "juguetes lego"  LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  4,8 de 5 estrellas 30.640  1 mil+ comprados el mes pasado24,01 €24,01€ PVPR: 29,99 €29,99€Entrega GRATIS el dom, 29 de sept en tu primer pedidoEntrega más rápida mañana, 27 de sept Vendido por: AmazonEdades: 4 años y más
LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  
4,8 de 5 estrellas
2
24.01€
Más vendidoen Figuras de dinosaurios y criaturas prehistóricas de juguete para niños  LEGO 31058 Creator Grandes Dinosaurios, Regalo de Reyes para Niños y Niñas de 7 Años o Más, Maqueta 3en1 de Pterodáctilo, Triceratops y T-Rex, Figura de Animales de Juguete de Parque Jurásico  4,7 de 5 estrellas 3.144  600+ compr

In [ ]:
productos[60].text.split('\n')

In [ ]:
print()


In [14]:
print(productos[0].prettify())

<div class="a-section a-spacing-base">
 <div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-v2fl5pkubaqu126k6zseo6li6q" data-cy="image-container" style="padding-top: 26px !important;">
  <span class="rush-component" data-component-type="s-product-image" data-render-id="rh0janz7ui9v42dur8tco6ab82" data-version-id="v2fl5pkubaqu126k6zseo6li6q">
   <a class="a-link-normal s-no-outline" href="/sspa/click?ie=UTF8&amp;spc=MTo1MDM0NTU3OTk1ODcwNjA0OjE3MjczNTQ5MTM6c3BfYXRmOjIwMDU1NjYwNzk0NTk4OjowOjo&amp;url=%2FKit-construccion-Fuertes-ni%25C3%25B1os-construcci%25C3%25B3n%2Fdp%2FB09KMY2WPR%2Fref%3Dsr_1_2_sspa%3F__mk_es_ES%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26crid%3D3GHAFETA0JY57%26dib%3DeyJ2IjoiMSJ9.JeseBEYIk1Lq23zhgXFBeLvR6IAQsidBwkDGe4FZrAkwxvV0nuk8HBhBHV9v0Erd2VdC1r4Y5d3D5VuC1jGMYjVIjkZCeQvzRwcPAngLvRu6qB1DQIWgQVEt0FssqYj8l0yloC

In [ ]:
#Vamos a sacar el nombre del producto, el precio , el link y las estrellas

for i,producto in enumerate(productos):
    try:
       nombre = producto.find_element(By.CLASS_NAME, 'a-size-base-plus').text
    except: # Exception as e:
        nombre = (f"El producto[{i}] sin nombre")#-->error: {e}
        print(nombre)

    try:
        precio = producto.find_element(By.CLASS_NAME, 'a-price-whole').text
    except:
        precio = (f"El producto[{i}] sin precio")
        print(precio)
    try:
        link = producto.find_element(By.CLASS_NAME, 'a-link-normal.a-text-normal').get_attribute('href')
    except:
        link = (f"El producto[{i}] sin link")
        print(link)
    #print(nombre, precio, link)

In [43]:
import pandas as pd
info_productos = []
for i, producto in enumerate(productos):
    dicc_prod = {}
    
    #Link del producto:
    prefijo_url = 'https://www.amazon.es'
    link_parcial = producto.find('a').attrs['href']
    link_producto = prefijo_url+link_parcial
    dicc_prod['link_producto'] = link_producto
    print(f"El producto {i} tiene el siguiente link:\n{link_producto}")
    
   
    #Nombre producto
    nombre = producto.find('h2').text
    dicc_prod['nombre'] = nombre
    print(f"El siguiente nombre:\n{nombre}")
    
    #Estrellas
    valoracion = float(producto.find('i').text.split(' ')[0].replace(',','.'))
    dicc_prod['valoracion'] = valoracion
    print(f"Las siguientes estrellas: {valoracion}")
    

    #Precio
    precio = float(producto.find('span', class_='a-price').text.split("€")[0].strip().replace(',','.'))
    dicc_prod['precio'] = precio
    print(f"El siguiente precio: {precio}€\n")
            
    info_productos.append(dicc_prod)


El producto 0 tiene el siguiente link:
https://www.amazon.es/LEGO-Classic-10696-ladrillos-creativos/dp/B00NVDP3ZU/ref=ice_ac_b_dpb?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3GHAFETA0JY57&dib=eyJ2IjoiMSJ9.JeseBEYIk1Lq23zhgXFBeLvR6IAQsidBwkDGe4FZrAkwxvV0nuk8HBhBHV9v0Erd2VdC1r4Y5d3D5VuC1jGMYjVIjkZCeQvzRwcPAngLvRu6qB1DQIWgQVEt0FssqYj8l0yloCK7FdsR3UQfSHLEoZpTgCCEbesF-Q5DazWiGQiLCtzZrWEl-uqCiZzl0FgyTlSlRAfbviAIWnLZSSmdpcexxg4JXMGROcea2d0apPbwP_GU-Ixqpp1TqKNr-GaZnR4pjeW3rAVW3k7lQxztW_gPxlgOKWCygHMxpLPCnv0._xb_FrfzyRK9PrbsD6bdK7XSe-mGD6xfCwHuLdF9ka0&dib_tag=se&keywords=juguetes+lego&qid=1727354913&sprefix=juguetes+lego%2Caps%2C108&sr=8-4
El siguiente nombre:
LEGO Classic Caja de Ladrillos Creativos Mediana, Juegos de Construcción para Niños y Niñas de 4 Años o Más, Base Verde, Coches y Animales de Juguete, Fácil Almacenaje, Idea de Regalo Infantil 10696  
Las siguientes estrellas: 4.8
El siguiente precio: 24.01€

El producto 1 tiene el siguiente link:
https://www.amazon.es/gp/bestsellers